In [ ]:
## montar o google driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [ ]:
import json
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from unsloth import FastLanguageModel
from trl import SFTTrainer

<ipython-input-3-406b103fd551>:5: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Configurações
DATA_DIR = "/content/drive/MyDrive/FIAP/tech_challange"
RAW_JSON = f"{DATA_DIR}/dataset_preparado_remove_outliers.json"
# RAW_JSONL = f"{DATA_DIR}/dataset_preparado_remove_outliers.jsonl"
MODEL_NAME_teste = "unsloth/Meta-Llama-3.1-8B"
MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 2048 ## Já rodei com 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
# 2. Carregar modelo e tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# 3. Aplicar LoRA
# model = FastLanguageModel.get_peft_model(
#     model,
#     r=32,
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
#     lora_alpha=64,
#     lora_dropout=0.2,
#     bias="none",
#     use_gradient_checkpointing="unsloth",
#     random_state=3407,
#     use_rslora=False,
#     loftq_config=None,
# )
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# 4. Preparação do dataset
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#     texts = []
#     for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
#         # Constrói o prompt com ou sem input
#         if input_text.strip():
#             user_prompt = f"{instruction}\n{input_text}"
#         else:
#             user_prompt = instruction

#         # Aplica formatação de chat (como LLaMA 3 espera)
#         formatted = (
#             f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
#             f"<|im_start|>assistant\n{output}{EOS_TOKEN}"
#         )
#         texts.append(formatted)
#     return { "text": texts }


dataset = load_dataset("json", data_files={"train": RAW_JSON}, split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)



In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
dtteste= dataset ##retirar

In [ ]:
dtteste.to_pandas().head()

,instruction,input,output,text
0,What is it Girls Ballet Tutu Neon Pink,,High quality 3 layer ballet tutu. 12 inches in...,"Below is an instruction that describes a task,..."
1,What is it Mog's Kittens,,Judith Kerr’s best–selling adventures of that ...,"Below is an instruction that describes a task,..."
2,What is it Girls Ballet Tutu Neon Blue,,Dance tutu for girls ages 2-8 years. Perfect f...,"Below is an instruction that describes a task,..."
3,What is it The Prophet,,"In a distant, timeless place, a mysterious pro...","Below is an instruction that describes a task,..."
4,What is it Rightly Dividing the Word,,--This text refers to thePaperbackedition.,"Below is an instruction that describes a task,..."


In [ ]:
print(dtteste[1])

{'instruction': "What is it Mog's Kittens", 'input': '', 'output': 'Judith Kerr’s best–selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books—with their bright, simple pictures, easy text, and hand–friendly formats—are just the thing to delight the very young. Ages 6 months–2 years.', 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is it Mog's Kittens\n\n### Input:\n\n\n### Response:\nJudith Kerr’s best–selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books—with their bright, simple pictures, easy text, and hand–friendly form

In [ ]:
# 5. Configuração e treinamento
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 1000,
        learning_rate = 2e-4, ##5e-5, ##2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", ##"linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc,
        warmup_ratio=0.1,
    ),
)

# trainer_stats = trainer.train()



Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
# 6. Estatísticas de GPU
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.135 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.460900
2,2.651100
3,2.838300
4,2.892300
5,2.838800
6,2.535600
7,2.675500
8,2.296500
9,2.589900
10,2.455800


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

8336.1726 seconds used for training.
138.94 minutes used for training.
Peak reserved memory = 7.756 GB.
Peak reserved memory for training = 0.621 GB.
Peak reserved memory % of max memory = 52.615 %.
Peak reserved memory for training % of max memory = 4.213 %.


In [ ]:
# 8. Função de inferência
def responder(instruction, input_text="", max_tokens=128):
    # Formatando o prompt com base na instrução e no input fornecido
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    prompt = alpaca_prompt.format(instruction, input_text, "")

    # Tokenizando a entrada e movendo para o dispositivo correto (GPU)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # Gerando a resposta com o modelo
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True)

    # Decodificando a saída para texto legível
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return resposta

# def responder(instruction, input_text="", max_tokens=128):
#     # Formatando o prompt com base na instrução e no input fornecido
#     alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# """

#     prompt = alpaca_prompt.format(instruction, input_text)

#     # Tokenizando a entrada e movendo para o dispositivo correto (GPU)
#     inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

#     # Gerando a resposta com o modelo
#     outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True)

#     # Decodificando a saída para texto legível
#     resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Removendo o cabeçalho da resposta gerada
#     resposta = resposta.split("### Response:")[-1].strip()

#     return resposta


In [ ]:
# Exemplos de uso
print(responder("O que é Windflower Wedding (Suttons of Yorkshire)?"))
print(responder("O que é The Prophet?"))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
O que é Windflower Wedding (Suttons of Yorkshire)?

### Input:


### Response:
"Fans of the series will enjoy this latest installment." --Library Journal
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
O que é The Prophet?

### Input:


### Response:
"O que é The Prophet?" is a short and simple introduction to Kahlil Gibran's famous collection of poetry.  It includes a brief biography, an explanation of the book's origins and a few pages of the original text.  The book's layout and illustrations are beautiful.  It's a great introduction to this classic work for young readers.


In [ ]:
# 7. Salvar modelo e tokenizer
model.save_pretrained(f"{DATA_DIR}/fine_tuned_model")
tokenizer.save_pretrained(f"{DATA_DIR}/fine_tuned_model")

('/content/drive/MyDrive/FIAP/tech_challange/fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/tech_challange/fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/tech_challange/fine_tuned_model/tokenizer.json')